# Data Preprocessing

## convert query responses to chat format

In [ ]:

import json 

def convert_single_turn_query_response_to_chat_format(path, split=''):
    data = []
    with open(path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    
    chat_data = []
    for ex in data:
        cur_chat = []
        cur_chat.append({'role': 'user', 'content': ex['query']})
        cur_chat.append({'role': 'assistant', 'content': ex['response']})
        if split == 'instructions' or split == 'dialog':
            chat_data.append({'messages': cur_chat, 'split': split, 'topic': ex['topic']})
        else:
            chat_data.append({'messages': cur_chat, 'split': split})
    return chat_data
    
    
non_comp_data_chat = convert_single_turn_query_response_to_chat_format('data/ft-v4/non_compliant_responses.json', split='non-compliant')
    
with open('data/ft-v5/non_compliant_all_chat.json', 'w') as f:
    for chat in non_comp_data_chat:
        f.write(json.dumps(chat) + '\n')
        

## data pruning

In [ ]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np


sim_model = SentenceTransformer("all-mpnet-base-v2")

def get_example_str(example, query_only=True):
    res = ""
    for m in example['messages']:
        content = m['content']
        if query_only:
            if m['role'] == 'user':
                res += f"{content}\n"
        else:
            res += f"{content}\n"
    return res


def prune_data(examples, threshold=0.9, query_only=True):
    print('initial data size: ', len(examples))
    
    np.random.shuffle(examples)
    
    pruned_data = []
    pruned_data_embs = None
    
    emb_dict = {}
    
    data_str = [get_example_str(d) for d in examples]
    data_emb = sim_model.encode(data_str, batch_size=256, show_progress_bar=True)
    
    # build sim dict
    for ex, emb in zip(data_str, data_emb):
        emb_dict[ex] = emb
    
    
    for ex in tqdm(examples):
        ex_str = get_example_str(ex)
        
        cur_emb = emb_dict[ex_str]
        
        if len(pruned_data) == 0:
            pruned_data.append(ex)
            pruned_data_embs = np.array([cur_emb])
            continue
            
        sim_mat = cur_emb.reshape(1, -1) @ pruned_data_embs.T
        max_sim = np.sort(sim_mat, axis=-1)[0, -1]
        
        if max_sim > threshold:
            continue
        
        pruned_data.append(ex)
        pruned_data_embs = np.concatenate([pruned_data_embs, cur_emb.reshape(1, -1)], axis=0)
        
    print("pruned data size: ", len(pruned_data))
    
    return pruned_data
    
_ = prune_data(non_comp_data_chat)


In [ ]:
non_comp_data_chat = prune_data(non_comp_data_chat, threshold=0.9)

with open('data/ft-v5/non_compliant_all_chat.json', 'w') as f:
    for chat in non_comp_data_chat:
        f.write(json.dumps(chat) + '\n')

## analyze single-turn instruction following data

In [ ]:
import json
import seaborn as sns

data = []
with open('data/ft-v4/gpt4o-instructions-12k.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
        
topics = [d['topic'] for d in data]

from collections import Counter

c = Counter(topics)

instructions = [d['query'] for d in data]

print(f"total instructions: {len(instructions)}")
print(f"total unique instructions: {len(set(instructions))}")


In [ ]:
with open('data/ft-v5/unique-gpt4o-instructions-12k.json', 'w') as f:
    cleaned_data = []
    queries = {}
    for ex in data:
        if ex['query'] not in queries:
            queries[ex['query']] = 1
            cleaned_data.append({'query': ex['query'], 'response': ex['response'], 'topic': ex['topic']})
    for ex in cleaned_data:
        f.write(json.dumps(ex) + '\n')
        
print(len(cleaned_data))

### visualize topics and sub-topics

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

df = pd.DataFrame(c.most_common(50), columns=['topic', 'count'])
plt.figure(figsize=(10,4))
sns.barplot(df, x='topic', y='count')
plt.xticks(rotation=90)
plt.title("top-50 topic counts")

In [ ]:
import re

def process_subtopics(subtopics_text):
    items = re.findall(r'^\d+\.\s*(.+)$', subtopics_text, re.MULTILINE)
    return [i.strip() for i in items]


process_subtopics(data[0]['subtopics'])

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as pyo

topics_dict = {k: Counter() for k in topics}

def process_subtopics(subtopics_text):
    items = re.findall(r'^\d+\.\s*(.+)$', subtopics_text, re.MULTILINE)
    return [i.strip() for i in items]

for d in data:
    subs = process_subtopics(d['subtopics'])
    topics_dict[d['topic']].update(subs)


# Set notebook mode to work in offline
# pyo.init_notebook_mode()
# Enable Plotly for Jupyter notebook
pio.renderers.default = 'png'

labels = []
parents = []
values = []

tc_mc = c.most_common(15)
topic_names = [t[0] for t in tc_mc]
# Add main topics
for topic, count in tc_mc:
    labels.append(topic)
    parents.append("")
    values.append(count)

for topic, subtopics in topics_dict.items():
    if topic not in topic_names:
        continue
    for subtopic, subcount in subtopics.most_common(5):
        
        labels.append(subtopic)
        parents.append(topic)
        values.append(subcount)

fig = go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
    # branchvalues="total",
))


fig.update_layout(
    margin=dict(t=0, l=0, r=0, b=0),
    width=800,  # Set the width here
    height=800 
)

# fig.write_image("topic-pie.png")
# Show the pl
fig.show()

## convert instruction response pairs to chat format

In [ ]:
instructions_chat = convert_single_turn_query_response_to_chat_format('data/ft-v6/gpt4o-instructions.json', split='instructions')
instructions_chat = prune_data(instructions_chat, threshold=0.9)


with open('data/ft-v6/gpt4o-instructions-chat-all.json', 'w') as f:
    for chat in instructions_chat:
        f.write(json.dumps(chat) + '\n')

## analyze dialog data

In [ ]:
import json

raw_data = []

with open('data/ft-v4/conversations-5k-raw.json', 'r') as f:
    for line in f:
        raw_data.append(json.loads(line))
        
raw_data[0]
    

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

topics = [d['topic'] for d in raw_data]

plt.figure(figsize=(10,4))
plt.xticks(rotation=90)
sns.histplot(topics)

In [ ]:
import re
from collections import Counter

def extract_scenarios(response):
    scenario_part = response.lower().split('<conversation>')[0].strip()
    pattern = r'\d+\.(\s+.*|^\d+\)\s+.*)'
    matches = re.findall(pattern, scenario_part)
    
    return [m.strip() for m in matches]    
    
    
topic_scenarios = {}
for ex in raw_data:
    scenarios = extract_scenarios(ex['response'])
    topic = ex['topic']
    
    if topic not in topic_scenarios:
        topic_scenarios[topic] = Counter()
    
    topic_scenarios[topic].update(scenarios)

topic_scenarios['Affordability'].most_common(20)

### post process conversations

In [ ]:
import re
from collections import Counter

def extract_conversations(response):
    split_token = '<conversation>' if '<conversation>' in response else '<Conversation>'
    conv_part = response.split(split_token)[1].strip()
    
    pattern = r"(User|Assistant): (.+?)(?=(User|Assistant): |$)"
    
    # Find all matches
    matches = re.findall(pattern, conv_part, re.DOTALL)
    
    # Create the list of messages in the required format
    hf_messages = []
    for role, message, _ in matches:
        hf_message = {
            "role": "user" if role == "User" else "assistant",
            "content": message.strip()
        }
        hf_messages.append(hf_message)
    
    return hf_messages
   

all_conversations = []
for ex in raw_data:
    all_conversations.append({'messages': extract_conversations(ex['response']), 'split': 'dialog', 'topic': ex['topic']})
    

In [ ]:
# distribution of conversation lengths

conv_lens = [len(c['messages']) for c in all_conversations]

sns.histplot(conv_lens, bins=np.arange(0, 18, 1))
plt.title("distribution of number of turns in dialog data")
plt.xlabel("number of turns")

### prune and save in chat format

In [ ]:
all_conversations = prune_data(all_conversations, threshold=0.9)

with open('data/ft-v5/conversations-chat-all.json', 'w') as f:
    for conv in all_conversations:
        f.write(json.dumps(conv) + '\n')

## adding ID and splitting data

In [ ]:
import json
from uuid import uuid4

non_comp_split = []
instruction_split = []
dialog_split = []

with open('data/ft-v5/non_compliant_all_chat.json', 'r') as f:
    for line in f:
        ex = json.loads(line)
        ex['id'] = str(uuid4())
        non_comp_split.append(ex)
        
with open('data/ft-v5/gpt4o-instructions-chat-all.json', 'r') as f:
    for line in f:
        ex = json.loads(line)
        ex['id'] = str(uuid4())
        instruction_split.append(ex)
        
with open('data/ft-v5/conversations-chat-all.json', 'r') as f:
    for line in f:
        ex = json.loads(line)
        ex['id'] = str(uuid4())
        dialog_split.append(ex)
        
print(f"total non compliant: {len(non_comp_split)}")
print(f"total instructions: {len(instruction_split)}")
print(f"total dialogues: {len(dialog_split)}")

In [ ]:
import numpy as np
# keeping 600 validation and 600 test examples 200 from each split

np.random.shuffle(non_comp_split)
np.random.shuffle(instruction_split)
np.random.shuffle(dialog_split)


non_comp_val = non_comp_split[:200]
non_comp_test = non_comp_split[200:400]
non_comp_train = non_comp_split[400:]

instruction_val = instruction_split[:200]
instruction_test = instruction_split[200:400]
instruction_train = instruction_split[400:]

dialog_val = dialog_split[:50]
dialog_train = dialog_split[50:]

print(f"non compliant: train: {len(non_comp_train)}, val: {len(non_comp_val)}, test: {len(non_comp_test)}")
print(f"instructions: train: {len(instruction_train)}, val: {len(instruction_val)}, test: {len(instruction_test)}")
print(f"dialogues: train: {len(dialog_train)}, val: {len(dialog_val)}")

In [ ]:

def write_jsonl(data, path):
    with open(path, 'w') as f:
        for ex in data:
            f.write(json.dumps(ex) + '\n')
            
non_comp_100p = non_comp_train
non_comp_50p = non_comp_train[:int(len(non_comp_train)*50/100)]
non_comp_25p = non_comp_train[:int(len(non_comp_train)*25/100)]
non_comp_75p = non_comp_train[:int(len(non_comp_train)*75/100)]


train_100p = non_comp_100p + instruction_train + dialog_train
train_50p = non_comp_50p + instruction_train + dialog_train
train_25p = non_comp_25p + instruction_train + dialog_train
train_75p = non_comp_75p + instruction_train + dialog_train

train_100p_single = non_comp_100p + instruction_train
train_50p_single = non_comp_50p + instruction_train
train_25p_single = non_comp_25p + instruction_train
train_75p_single = non_comp_75p + instruction_train

version = 'ft-v5'

write_jsonl(non_comp_train, f'data/{version}/non_compliant_train.json')
write_jsonl(non_comp_val, f'data/{version}/non_compliant_val.json')
write_jsonl(non_comp_test, f'data/{version}/non_compliant_test.json')
write_jsonl(instruction_train, f'data/{version}/instruction_train.json')
write_jsonl(instruction_val, f'data/{version}/instruction_val.json')
write_jsonl(instruction_test, f'data/{version}/instruction_test.json')
write_jsonl(dialog_train, f'data/{version}/dialog_train.json')
write_jsonl(dialog_val, f'data/{version}/dialog_val.json')
# write_jsonl(dialog_test, f'data/{version}/dialog_test.json')

write_jsonl(train_100p, f'data/{version}/train_100p.json')
write_jsonl(train_50p, f'data/{version}/train_50p.json')
write_jsonl(train_25p, f'data/{version}/train_25p.json')
write_jsonl(train_75p, f'data/{version}/train_75p.json')

write_jsonl(train_100p_single, f'data/{version}/train_100p_single.json')
write_jsonl(train_50p_single, f'data/{version}/train_50p_single.json')
write_jsonl(train_25p_single, f'data/{version}/train_25p_single.json')
write_jsonl(train_75p_single, f'data/{version}/train_75p_single.json')

all_val = non_comp_val + instruction_val + dialog_val
write_jsonl(all_val, f'data/{version}/validation_all.json')


# Nearest neighbor analysis of data

In [ ]:
import json


def load_jsonl(path):
    data = []
    with open(path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data


def convert_to_string_chat(example, only_query=False):
    chat_str = ""
    for turn in example['messages']:
        if not only_query:
            chat_str = chat_str + f"{turn['role']}: {turn['content']}\n\n"
        else:
            if turn['role'] == 'user':
                chat_str += f"{turn['content']}\n"
    return chat_str

# instruction_data = load_jsonl('data/ft-v5/instruction_train.json')
# instruction_data = [convert_to_string_chat(x) for x in instruction_data]

# instruction_test_data = load_jsonl('data/ft-v5/instruction_test.json')
# instruction_test_data = [convert_to_string_chat(x) for x in instruction_test_data]


## similarity between training samples

In [ ]:
from sentence_transformers import SentenceTransformer
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

model = SentenceTransformer("all-mpnet-base-v2")
# non_comp_data = load_jsonl('data/ft-v5/non_compliant_train.json')
inst_data = load_jsonl('data/ft-v6/gpt4o-instructions-chat-all.json')
# dialog_data = load_jsonl('data/ft-v5/dialog_train.json')

def plot_nn_sim(data, title="", only_query=False):
    print("data size: ", len(data))
    data = [convert_to_string_chat(x, only_query=only_query) for x in data]
    embeddings = model.encode(data, batch_size=128, show_progress_bar=True)
    similarity_mat = embeddings @ embeddings.T
    print("similarity shape:", similarity_mat.shape)
    nn_similarity = np.sort(similarity_mat, axis=1)[:, -2]
    

    sns.histplot(nn_similarity)
    plt.title(title)
    plt.show()

# plot_nn_sim(non_comp_data, title='NN similarity between non-comp training samples for question response pairs')
plot_nn_sim(inst_data, title='NN similarity between instruction training samples for question response pairs')
# plot_nn_sim(dialog_data, title='NN similarity between dialog training samples for question response pairs')

# plot_nn_sim(non_comp_data, title='NN similarity between non-comp training samples for questions', only_query=True)
plot_nn_sim(inst_data, title='NN similarity between instruction training samples for questions', only_query=True)
# plot_nn_sim(dialog_data, title='NN similarity between dialog training samples for questions', only_query=True)

## similarity of train and test 

In [ ]:
import numpy as np

embeddings = model.encode(instruction_data, batch_size=128, show_progress_bar=True)
test_embeddings = model.encode(instruction_test_data, batch_size=128, show_progress_bar=True)


similarity_mat = test_embeddings @ embeddings.T
similarity_mat.shape

In [ ]:
nn_similarity = np.sort(similarity_mat, axis=1)[:, -1]
nn_similarity[:10]

In [ ]:
import seaborn as sns

sns.histplot(nn_similarity, bins=np.arange(0.4, 1.0, 0.01))